In [8]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import librosa
import librosa.display
from tqdm import tqdm

EMOTIONS = {1:"neu",2:'cal',3:'hap',4:'sad',5:'ang',6:'fea',7:'dis',8:'sur'}
EMOTION_NUM = 8
TILTLEEMOTIONS={"neu":'中性','cal':'平静','hap':'快乐','sad':'悲伤','ang':'生气','fea':'害怕','dis':'厌恶','sur':'惊讶'}
SOURCE_PATH = "db/RAVDESS/"
SAMPLE_RATE = 48000
EPOCH=10
FILE_PATH=[]
DURATION=3
OFFSET=0.5

df = pd.DataFrame(columns = ['FILE','EMOTION',])
FILE_NAME=[]
EMOTIONLIST=[]
for director, _, file_names in os.walk(SOURCE_PATH):
    for file_name in file_names:
        file_path = os.path.join(director+'/',file_name)
        FILE_NAME.append(file_path)
        EMOTIONLIST.append(file_name.split('.')[0].split('-')[2])
df["FILE"]=FILE_NAME
df["EMOTION"]=EMOTIONLIST

print("共有{}行".format(df.shape[0]))
df.head(n=10)

共有1440行


,FILE,EMOTION
0,db/RAVDESS/Actor_01/03-01-01-01-01-01-01.wav,01
1,db/RAVDESS/Actor_01/03-01-01-01-01-02-01.wav,01
2,db/RAVDESS/Actor_01/03-01-01-01-02-01-01.wav,01
3,db/RAVDESS/Actor_01/03-01-01-01-02-02-01.wav,01
4,db/RAVDESS/Actor_01/03-01-02-01-01-01-01.wav,02
5,db/RAVDESS/Actor_01/03-01-02-01-01-02-01.wav,02
6,db/RAVDESS/Actor_01/03-01-02-01-02-01-01.wav,02
7,db/RAVDESS/Actor_01/03-01-02-01-02-02-01.wav,02
8,db/RAVDESS/Actor_01/03-01-02-02-01-01-01.wav,02
9,db/RAVDESS/Actor_01/03-01-02-02-01-02-01.wav,02


In [20]:
from sklearn.model_selection import train_test_split

train_index,test_eval_index=train_test_split(list(df.index),test_size=0.2,random_state=1)
test_index,eval_index=train_test_split(test_eval_index,test_size=0.5,random_state=1)

print("train_index前10个:\t",train_index[:10])
print("eval_index前10个:\t",eval_index[:10])
print("test_index前十个:\t",test_index[:10])
print('train_index/(train_index+test_eval_index)={}/({}+{})={}'.format(len(train_index),len(train_index),len(test_eval_index),len(train_index)/(len(train_index)+len(test_eval_index))))
print('eval_index/(test_index+eval_index)={}/({}+{})={}'.format(len(eval_index),len(test_index),len(eval_index),len(eval_index)/(len(eval_index)+len(test_index))))


train_index前10个:	 [135, 368, 750, 801, 1322, 1060, 613, 994, 527, 484]
eval_index前10个:	 [1092, 495, 75, 1398, 215, 1108, 1279, 223, 735, 1053]
test_index前十个:	 [65, 1093, 925, 1170, 536, 675, 1417, 921, 1081, 1383]
train_index/(train_index+test_eval_index)=1152/(1152+288)=0.8
eval_index/(test_index+eval_index)=144/(144+144)=0.5


In [17]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def getMELspec(data,sr):
    # shape=(n_mels, t)
    mel_spec = librosa.feature.melspectrogram(y = data,
                                              sr = SAMPLE_RATE,
                                              n_fft=1024,      # length of the FFT window
                                              win_length=1024,
                                              window='hamming',
                                              hop_length=512,
                                              n_mels=64
                                             )
    mel_spec_db = librosa.power_to_db(mel_spec,ref=np.max)
    mfccs = librosa.feature.mfcc(S=mel_spec_db)
    return mel_spec_db,mfccs

y_data=np.array(df['EMOTION'])
y_data=y_data.astype(np.int32)
y_data=y_data-1

alpha = 0.97
def PreEmphsised(data):
    return np.append(data[0],data[1:] - alpha * data[:-1])

def sameLenData(data):
    smLenData = np.zeros(SAMPLE_RATE*DURATION)
    smLenData[0:len(data)]=data
    return smLenData


def get3DMel(data):#model\\model2_1.pt  65%
    shape=data.shape
    diff_1=np.zeros(shape)
    diff_2=np.zeros(shape)
    diff_1[:,:shape[1]-1] = np.diff(data,n=1)
    diff_2[:,:shape[1]-2] = np.diff(data,n=2)
    return diff_1,diff_2,[data,diff_1 ,diff_2]

In [4]:
# BITS_LEN=16
# AUG_NUM=2
# SNR_LO=15
# SNR_HI=30
# def AWGN(data):
#     data_len=len(data)
#     noise=np.random.normal(size=(AUG_NUM,data_len))
#     norm_con=2**(BITS_LEN-1)
#     signal_norm = data/norm_con
#     noise_norm = noise/norm_con
#     s_power=np.sum(signal_norm**2)/data_len
#     n_power=np.sum(noise_norm**2)/data_len
#     target_snr=np.random.randint(SNR_LO,SNR_HI)
#     K=np.sqrt((s_power/n_power)*10**(-target_snr/10))
#     K=np.ones((data_len,AUG_NUM))*K
#     return data+K.T*noise
    
# arg_data=[]
# arg_label=[]
# arg_index=[]
# start=max(list(df.index))+1
# sr=0
# with tqdm(total=len(train_index),desc="数据加载") as loadbar:
#     for i in train_index:
#         data, sr = librosa.load(df['FILE'][i],sr=SAMPLE_RATE,duration=DURATION,offset=OFFSET)
#         arg=AWGN(data)
#         x_1,_=getMELspec(sameLenData(arg[0]),sr)
#         x_2,_=getMELspec(sameLenData(arg[1]),sr)
#         arg_data.append(x_1)
#         arg_data.append(x_2)
#         arg_label.append(y_data[i])
#         arg_label.append(y_data[i])
#         loadbar.update(1)

数据加载: 100%|████████████████████████████████████████████████████████████████████| 1152/1152 [01:21<00:00, 14.06it/s]


In [21]:
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import torch

scaler = StandardScaler()
DURATION=3
OFFSET=0.5
class SpeechDataset(Dataset):
    def __init__(self,index,setName=None):
        self.x_data = []
        self.arg = np.zeros([1,144000])
        self.y_data = []
        with tqdm(total=len(index)+1,desc="数据加载") as loadbar:
            for i in index: # 写入1440个文件的数据
                data, sr = librosa.load(df['FILE'][i],sr=SAMPLE_RATE,duration=DURATION,offset=OFFSET)
                smLenData = sameLenData(data)
                # 预加重
                smLenData = PreEmphsised(smLenData)
                # 特征向量
                mel_data,mfcc_data = getMELspec(smLenData,sr)
                # 差分向量
                #_,_,mel_data = get3DMel(mel_data)
                self.x_data.append(list(mel_data))
                self.y_data.append(y_data[i])
                loadbar.update(1)
#             if setName=='train_index':
#                 self.x_data=np.concatenate([self.x_data,arg_data],axis=0)
#                 self.y_data=np.concatenate([self.y_data,arg_label],axis=0)
            self.x_data=np.expand_dims(self.x_data,1)
            shape=np.array(self.x_data).shape
            self.x_data = np.reshape(self.x_data,newshape=(shape[0],-1))
            if setName=='train_index':
                self.x_data = scaler.fit_transform(self.x_data)
            else:
                self.x_data = scaler.transform(self.x_data)
            self.x_data=np.reshape(self.x_data,newshape=shape)
            self.x_data=torch.from_numpy(np.array(self.x_data,dtype=np.float64))
            self.y_data=torch.from_numpy(np.array(self.y_data,dtype=np.float64))
  
            self.len = len(index)
            loadbar.update(1)
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len


In [22]:
BATCH_SIZE=32

# 训练集加载
train_dataset = SpeechDataset(train_index,'train_index')
train_loader = DataLoader(train_dataset,
                          shuffle=True,
                          batch_size=BATCH_SIZE,
                         )
# 验证集加载
eval_dataset = SpeechDataset(eval_index)
eval_loader = DataLoader(eval_dataset,
                          shuffle=False,
                          batch_size=BATCH_SIZE,
                         )
# 测试集加载
test_dataset = SpeechDataset(test_index)
test_loader = DataLoader(test_dataset,
                          shuffle=False,
                          batch_size=BATCH_SIZE,
                         )


数据加载: 100%|██████████████████████████████████████████████████████████████████████| 145/145 [00:04<00:00, 29.77it/s]


In [24]:
print(train_dataset.x_data.shape)
print(eval_dataset.x_data.shape)
print(test_dataset.x_data.shape)

torch.Size([1152, 1, 64, 282])
torch.Size([144, 1, 64, 282])
torch.Size([144, 1, 64, 282])


In [36]:
print('x_data:',train_dataset.x_data.shape,'  y_data',train_dataset.y_data.shape)
#np.set_printoptions(threshold=np.inf)
print(np.array(train_dataset.x_data[0]))
print(np.array(eval_dataset.x_data[0]))
print(np.array(test_dataset.x_data[0]))

x_data: torch.Size([1152, 1, 64, 282])   y_data torch.Size([1152])
[[[-0.67325471 -0.6821769  -0.69511429 ... -0.63889715 -0.64009395
   -0.12269867]
  [-0.62930198 -0.59434941 -0.60018943 ... -0.55454918 -0.56611444
    0.14377254]
  [-0.61070813 -0.5604981  -0.57121697 ... -0.5390266  -0.5435462
   -0.06885969]
  ...
  [-1.04204697 -1.06620386 -1.07144973 ...  0.01656576 -0.19195955
   -0.21951804]
  [-1.0418595  -1.07270438 -1.07460435 ... -0.04612548 -0.30882887
   -0.15334563]
  [-1.03902234 -1.06338046 -1.07141052 ... -0.09199534 -0.34217952
   -0.38634281]]]
[[[2.15082589 0.50382796 0.43699593 ... 3.69490294 3.83674473 3.89645083]
  [2.38720546 1.01889965 1.01030396 ... 1.4825624  1.45573554 1.25276435]
  [1.89543636 1.08976406 1.02939147 ... 0.51136153 0.52635476 0.48685327]
  ...
  [2.03571829 1.45808851 1.78926593 ... 2.24626856 2.29379796 2.1371975 ]
  [2.08007865 1.46623807 1.63826211 ... 2.16340767 2.16687631 2.19449819]
  [2.16775224 1.86209643 1.72413553 ... 2.14209925 2

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class attention(nn.Module):
    def __init__(self,en_hidden_dim,de_hidden_dim,direction):
        super().__init__()
        middle_size = de_hidden_dim
        # H与s0拼接出来为(batch_size,src_len,en_hidden_dim*direction+de_hidden_dim)
        # 需要转化维度。首先，a(batch_size,src_len),公式2得知v(batch_size,?),E(?,src_len)
        # 因此?可以是任意维度，此处取de_hidden_dim
        self.s_en2de=nn.Linear(en_hidden_dim*2,de_hidden_dim)
        self.attn=nn.Linear(en_hidden_dim*direction+de_hidden_dim,middle_size)
        self.v = nn.Linear(middle_size,1)
        
    def forward(self,gru_output,gru_hidden):       
        # H(batch_size,src_len,en_hidden_dim*direction)
        _,src_len,_=gru_output.shape
        # s0(batch_size,en_hidden_dim*direction)
        # 但是s0作为dcoder初始隐藏状态，应该是s0(batch_size,de_hidden_dim)
        # 并且为了与H拼接，需要加一个维度，变成s0(batch_size,src_len,de_hidden_dim)
        s = torch.cat((gru_hidden[:, -2,: ], gru_hidden[:, -1,:]),dim=1)  # (batch_size,en_hidden_dim*direction)
        s = self.s_en2de(s)  # (batch_size,de_hidden_dim)
        # s=torc.tanh(s)  
        s=s.unsqueeze(1).repeat(1,src_len,1) # (batch_size,src_len,de_hidden_dim)     
        self.attn_hidden = torch.tanh(self.attn(torch.cat((s,gru_output),dim=2)))     
        attention_weight = self.v(self.attn_hidden).squeeze(2) #(batch_size,1,src_len)
        
        return F.softmax(attention_weight,dim=1)   #A dimension along which Softmax will be computed (so every slice along dim will sum to 1

In [85]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# class attention(nn.Module):
#     def __init__(self,en_hidden_dim,de_hidden_dim,direction):
#         super().__init__()
#         self.linear = nn.Linear(direction*en_hidden_dim,1)
#     def forward(self,gru_output,gru_hidden):
#         shape=gru_output.shape
#         attention = [None]*shape[1]
#         for t in range(shape[1]):
#             attention[t] = self.linear(gru_output[:,t,:])
#         soft_attention_weight = nn.functional.softmax(torch.stack(attention,-1),-1)
#         return soft_attention_weight

In [38]:
class ResidualBlock(torch.nn.Module):
    def __init__(self,channels):
        super(ResidualBlock, self).__init__()
        self.channels = channels
        self.conv1 = torch.nn.Conv2d(channels,channels,kernel_size=3,padding=1)
        self.conv2 = torch.nn.Conv2d(channels, channels, kernel_size=3, padding=1)
    def forward(self,x):
        y = self.conv2(F.relu(self.conv1(x)))
        return  F.relu(x + y)

In [28]:
hidden_size,input_size,p=128,16,0.3
kernel_size,stride=4,4
directional = 2

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        """
         input dimension every epoch is [32,1,128,282]
        """
        # conv2d
#         self.conv2d=nn.Sequential(# No 1
#                                    nn.Conv2d(in_channels=1,out_channels=8,kernel_size=3,stride=1,padding=1)
#                                   ,nn.BatchNorm2d(8)
#                                   ,nn.ReLU()
#                                   ,nn.MaxPool2d(kernel_size=2,stride=2)
#                                   ,nn.Dropout(p)
                                    
#                                   #,Inception(8)  #outchannel 88     35840
                                  
#                                   ,ResidualBlock(8)
            
#                                   ,nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3,stride=1,padding=1)
#                                   ,nn.BatchNorm2d(128)
#                                   ,nn.ReLU()
#                                   ,nn.MaxPool2d(kernel_size=2,stride=2)
#                                   ,nn.Dropout(p)
#         )
        self.conv2d=nn.Sequential(# No 1
                                   nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3,stride=1,padding=1)
                                  ,nn.BatchNorm2d(16)
                                  ,nn.ReLU()
                                  ,nn.MaxPool2d(kernel_size=2,stride=2)
                                  ,nn.Dropout(p)
            
                                  # No 2
                                  ,nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1,padding=1)
                                  ,nn.BatchNorm2d(32)
                                  ,nn.ReLU()
                                  ,nn.MaxPool2d(kernel_size=2,stride=2)
                                  ,nn.Dropout(p)
            
                                  # No 3
                                  ,nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1)
                                  ,nn.BatchNorm2d(64)
                                  ,nn.ReLU()
                                  ,nn.MaxPool2d(kernel_size=4,stride=4)
                                  ,nn.Dropout(p)
            
                                  # No 4
                                  ,nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,stride=1,padding=1)
                                  ,nn.BatchNorm2d(64)
                                  ,nn.ReLU()
                                  ,nn.MaxPool2d(kernel_size=2,stride=2)
                                  ,nn.Dropout(p)
        )
        
        # GRU
        self.mp = nn.MaxPool2d(kernel_size=kernel_size,stride=stride)
        self.gru = nn.GRU(input_size=input_size,hidden_size=hidden_size,bidirectional =bool(directional-1),batch_first=True)
        self.dp = nn.Dropout(p)
        
        self.attention = attention(hidden_size,hidden_size,directional)
        self.emo_linear=nn.Linear(hidden_size*directional+1024,EMOTION_NUM) 
       
        self.out_dropout = nn.Dropout(0)
        
    def forward(self,x):
        x=x.float()
        
        # conv
        #convx=x[:,1].unsqueeze(1)
        convx = self.conv2d(x)
        convx = torch.flatten(convx,1) 
        #print(convx.shape)
        
        #GRU
        x = self.mp(x)   # input x:(N,C,freq,time)
        x = torch.squeeze(x)
        x = x.permute(0,2,1)  # batch_size, time, freq
        #x = torch.cat([x[:,0],x[:,1],x[:,2]],dim=2)
        
        # gru_output(batch_size, sequence length, hidden_size*directional)
        # gru_h(batch_size,n_layer*directional,hidden_size)
        # gru_h=[for_1,back_1,for_2,back_2...] 
        
        gru_output,gru_h=self.gru(x) # gru_output(B, S, H*2), gru_h(n_layer*directional,B,H)
        gru_h=gru_h.permute(1,0,2)
        gru_output = self.dp(gru_output)
        attention_weight = self.attention(gru_output,gru_h) # (batch_size,1,src_len)
        
        # (batch_size,1,src_len)*(batch_size,src_len,hidden_size*directional)
        # (batch_size,1,hidden_size*directional)
        attention_weight = attention_weight.unsqueeze(1)
#         print(attention_weight.shape)
#         print(gru_output.shape)
        attention = torch.bmm(attention_weight,gru_output)
        attention = torch.squeeze(attention,1) #（batch_size,hidden_size*directional）
        
        # cat
        output = torch.cat([convx,attention],dim=1)
        #print(attention.shape)
        #print(output.shape)
        output = self.out_dropout(self.emo_linear(output))
        return output

In [29]:
model = Model()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01,weight_decay=1e-3,momentum=0.5)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

loss_set=[]
accuracy_train=[]
accuracy_validate=[]
def train(epoch):
    running_loss = 0.0
    total = 0
    correct = 0
    for batch_idx,(input,target) in enumerate(train_loader,0):
        input,target = input.to(device),target.to(device)
        model.train()
        optimizer.zero_grad()
        output=model(input)
        _,pre = torch.max(output.data,dim=1)
        correct += (pre==target).sum().item()
        total += target.size(0)
        target = target.long()
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        loss_set.append(loss.item())
        if batch_idx % 20 == 19:
            print('[%d,%5d] loss: %.5f, running_loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / min((batch_idx+1)*BATCH_SIZE,df.shape[0]), running_loss))
            running_loss = 0
    loss_set.append(running_loss)
    accuracy_train.append(correct / total)
    print('Accacy on train_loader set: %d %% [%d/%d]' % (100 * correct / total, correct, total))   
      
#total = 0
#correct = 0
def validate(loader):
    total = 0
    correct = 0
    for (input,target) in (eval_loader if loader=='eval_loader' else test_loader):
        model.eval()
        input,target  =input.to(device),target.to(device)
        output = model(input)
        _,pre = torch.max(output.data,dim=1)
        correct += (pre==target).sum().item()
        total += target.size(0)  
    currect_rate = correct / total
    print('Accacy on %s set: %d %% [%d/%d]' % (loader,100 * correct / total, correct, total)) 
    accuracy_validate.append(currect_rate)
    return currect_rate

cuda:0


In [14]:
model_path='model\\model3_1.pt' 
opti_path='model\\opti3_1.pt' 
loss_path='model\\loss3_1.txt'
accur_train_path='model\\accur_train3_1.txt'
accur_valid_path='model\\accur_valid3_1.txt'
def saveModel(model_path):
    torch.save(model.state_dict(),model_path)
    torch.save(optimizer.state_dict(),opti_path) 
#     print("Model's state_dict:")
#     for param_tensor in model.state_dict():
#         print(param_tensor, "\t", model.state_dict()[param_tensor].size())
        
def loadModel(model_path):
    model = Model()
    model.to(device)
    model.load_state_dict(torch.load(model_path),strict=False)
    return model

def saveData(loss,accuracy_train,accuracy_validate):
    np.savetxt(loss_path,np.array(loss),fmt='%0.8f')
    np.savetxt(accur_train_path,np.array(accuracy_train),fmt='%0.8f')
    np.savetxt(accur_valid_path,np.array(accuracy_validate),fmt='%0.8f')

In [103]:
if __name__ == '__main__':
    max_correct_rate=0.0
    for epoch in range(1500):
        train(epoch)
        correct_rate=validate('eval_loader')
        if max_correct_rate < correct_rate:
            max_correct_rate = correct_rate
            saveModel(model_path)
        

[1,   20] loss: 0.07075, running_loss: 45.280
Accacy on train_loader set: 22 % [263/1152]
Accacy on eval_loader set: 15 % [22/144]
[2,   20] loss: 0.05896, running_loss: 37.736
Accacy on train_loader set: 32 % [371/1152]
Accacy on eval_loader set: 16 % [24/144]
[3,   20] loss: 0.05289, running_loss: 33.850
Accacy on train_loader set: 38 % [438/1152]
Accacy on eval_loader set: 15 % [23/144]
[4,   20] loss: 0.04961, running_loss: 31.752
Accacy on train_loader set: 40 % [472/1152]
Accacy on eval_loader set: 15 % [22/144]
[5,   20] loss: 0.04795, running_loss: 30.689
Accacy on train_loader set: 42 % [484/1152]
Accacy on eval_loader set: 18 % [26/144]
[6,   20] loss: 0.04790, running_loss: 30.659
Accacy on train_loader set: 43 % [500/1152]
Accacy on eval_loader set: 19 % [28/144]
[7,   20] loss: 0.04741, running_loss: 30.342
Accacy on train_loader set: 44 % [507/1152]
Accacy on eval_loader set: 20 % [29/144]
[8,   20] loss: 0.04338, running_loss: 27.765
Accacy on train_loader set: 45 % [523

Accacy on train_loader set: 73 % [841/1152]
Accacy on eval_loader set: 47 % [68/144]
[64,   20] loss: 0.02347, running_loss: 15.018
Accacy on train_loader set: 72 % [836/1152]
Accacy on eval_loader set: 52 % [75/144]
[65,   20] loss: 0.02174, running_loss: 13.914
Accacy on train_loader set: 71 % [824/1152]
Accacy on eval_loader set: 50 % [73/144]
[66,   20] loss: 0.02245, running_loss: 14.369
Accacy on train_loader set: 73 % [844/1152]
Accacy on eval_loader set: 55 % [80/144]
[67,   20] loss: 0.02425, running_loss: 15.520
Accacy on train_loader set: 75 % [870/1152]
Accacy on eval_loader set: 52 % [75/144]
[68,   20] loss: 0.02271, running_loss: 14.536
Accacy on train_loader set: 73 % [847/1152]
Accacy on eval_loader set: 51 % [74/144]
[69,   20] loss: 0.02119, running_loss: 13.559
Accacy on train_loader set: 73 % [843/1152]
Accacy on eval_loader set: 47 % [69/144]
[70,   20] loss: 0.02274, running_loss: 14.551
Accacy on train_loader set: 74 % [858/1152]
Accacy on eval_loader set: 50 % 

Accacy on train_loader set: 84 % [974/1152]
Accacy on eval_loader set: 57 % [83/144]
[126,   20] loss: 0.01549, running_loss: 9.914
Accacy on train_loader set: 82 % [953/1152]
Accacy on eval_loader set: 61 % [89/144]
[127,   20] loss: 0.01440, running_loss: 9.219
Accacy on train_loader set: 84 % [975/1152]
Accacy on eval_loader set: 54 % [79/144]
[128,   20] loss: 0.01288, running_loss: 8.246
Accacy on train_loader set: 85 % [983/1152]
Accacy on eval_loader set: 56 % [81/144]
[129,   20] loss: 0.01324, running_loss: 8.471
Accacy on train_loader set: 85 % [987/1152]
Accacy on eval_loader set: 59 % [85/144]
[130,   20] loss: 0.01369, running_loss: 8.761
Accacy on train_loader set: 84 % [971/1152]
Accacy on eval_loader set: 56 % [81/144]
[131,   20] loss: 0.01392, running_loss: 8.906
Accacy on train_loader set: 84 % [974/1152]
Accacy on eval_loader set: 56 % [82/144]
[132,   20] loss: 0.01498, running_loss: 9.588
Accacy on train_loader set: 83 % [958/1152]
Accacy on eval_loader set: 55 % 

Accacy on train_loader set: 90 % [1040/1152]
Accacy on eval_loader set: 65 % [94/144]
[188,   20] loss: 0.00879, running_loss: 5.624
Accacy on train_loader set: 90 % [1040/1152]
Accacy on eval_loader set: 65 % [94/144]
[189,   20] loss: 0.00824, running_loss: 5.272
Accacy on train_loader set: 90 % [1038/1152]
Accacy on eval_loader set: 64 % [93/144]
[190,   20] loss: 0.00900, running_loss: 5.758
Accacy on train_loader set: 90 % [1039/1152]
Accacy on eval_loader set: 61 % [88/144]
[191,   20] loss: 0.00768, running_loss: 4.913
Accacy on train_loader set: 92 % [1062/1152]
Accacy on eval_loader set: 61 % [89/144]
[192,   20] loss: 0.00886, running_loss: 5.672
Accacy on train_loader set: 89 % [1030/1152]
Accacy on eval_loader set: 55 % [80/144]
[193,   20] loss: 0.00917, running_loss: 5.868
Accacy on train_loader set: 90 % [1037/1152]
Accacy on eval_loader set: 62 % [90/144]
[194,   20] loss: 0.00789, running_loss: 5.051
Accacy on train_loader set: 91 % [1059/1152]
Accacy on eval_loader se

[249,   20] loss: 0.00561, running_loss: 3.592
Accacy on train_loader set: 94 % [1089/1152]
Accacy on eval_loader set: 61 % [89/144]
[250,   20] loss: 0.00626, running_loss: 4.005
Accacy on train_loader set: 93 % [1081/1152]
Accacy on eval_loader set: 63 % [92/144]
[251,   20] loss: 0.00712, running_loss: 4.555
Accacy on train_loader set: 92 % [1067/1152]
Accacy on eval_loader set: 54 % [79/144]
[252,   20] loss: 0.00701, running_loss: 4.489
Accacy on train_loader set: 92 % [1070/1152]
Accacy on eval_loader set: 67 % [97/144]
[253,   20] loss: 0.00639, running_loss: 4.090
Accacy on train_loader set: 93 % [1079/1152]
Accacy on eval_loader set: 63 % [92/144]
[254,   20] loss: 0.00713, running_loss: 4.561
Accacy on train_loader set: 92 % [1066/1152]
Accacy on eval_loader set: 61 % [89/144]
[255,   20] loss: 0.00619, running_loss: 3.962
Accacy on train_loader set: 93 % [1081/1152]
Accacy on eval_loader set: 56 % [82/144]
[256,   20] loss: 0.00443, running_loss: 2.838
Accacy on train_loader

...

Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 75 % [108/144]
[1234,   20] loss: 0.00154, running_loss: 0.984
Accacy on train_loader set: 98 % [1140/1152]
Accacy on eval_loader set: 71 % [103/144]
[1235,   20] loss: 0.00106, running_loss: 0.680
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 75 % [108/144]
[1236,   20] loss: 0.00119, running_loss: 0.760
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 70 % [101/144]
[1237,   20] loss: 0.00110, running_loss: 0.702
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 74 % [107/144]
[1238,   20] loss: 0.00138, running_loss: 0.884
Accacy on train_loader set: 98 % [1139/1152]
Accacy on eval_loader set: 70 % [102/144]
[1239,   20] loss: 0.00132, running_loss: 0.848
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 73 % [106/144]
[1240,   20] loss: 0.00080, running_loss: 0.510
Accacy on train_loader set: 99 % [1144/1152]
Accacy on 

Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 72 % [104/144]
[1295,   20] loss: 0.00119, running_loss: 0.761
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 73 % [106/144]
[1296,   20] loss: 0.00142, running_loss: 0.912
Accacy on train_loader set: 98 % [1137/1152]
Accacy on eval_loader set: 72 % [105/144]
[1297,   20] loss: 0.00095, running_loss: 0.609
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 73 % [106/144]
[1298,   20] loss: 0.00128, running_loss: 0.821
Accacy on train_loader set: 98 % [1140/1152]
Accacy on eval_loader set: 72 % [105/144]
[1299,   20] loss: 0.00084, running_loss: 0.535
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 71 % [103/144]
[1300,   20] loss: 0.00139, running_loss: 0.889
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 73 % [106/144]
[1301,   20] loss: 0.00119, running_loss: 0.762
Accacy on train_loader set: 99 % [1141/1152]
Accacy on 

Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 74 % [107/144]
[1356,   20] loss: 0.00150, running_loss: 0.960
Accacy on train_loader set: 98 % [1137/1152]
Accacy on eval_loader set: 77 % [111/144]
[1357,   20] loss: 0.00144, running_loss: 0.923
Accacy on train_loader set: 98 % [1137/1152]
Accacy on eval_loader set: 72 % [105/144]
[1358,   20] loss: 0.00174, running_loss: 1.114
Accacy on train_loader set: 98 % [1138/1152]
Accacy on eval_loader set: 75 % [108/144]
[1359,   20] loss: 0.00136, running_loss: 0.872
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 70 % [101/144]
[1360,   20] loss: 0.00104, running_loss: 0.666
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 70 % [101/144]
[1361,   20] loss: 0.00083, running_loss: 0.529
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 71 % [103/144]
[1362,   20] loss: 0.00130, running_loss: 0.834
Accacy on train_loader set: 99 % [1142/1152]
Accacy on 

Accacy on train_loader set: 98 % [1137/1152]
Accacy on eval_loader set: 72 % [104/144]
[1417,   20] loss: 0.00136, running_loss: 0.870
Accacy on train_loader set: 98 % [1135/1152]
Accacy on eval_loader set: 70 % [101/144]
[1418,   20] loss: 0.00161, running_loss: 1.029
Accacy on train_loader set: 98 % [1135/1152]
Accacy on eval_loader set: 71 % [103/144]
[1419,   20] loss: 0.00098, running_loss: 0.629
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 72 % [105/144]
[1420,   20] loss: 0.00123, running_loss: 0.788
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 69 % [100/144]
[1421,   20] loss: 0.00084, running_loss: 0.540
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 68 % [98/144]
[1422,   20] loss: 0.00079, running_loss: 0.508
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 69 % [100/144]
[1423,   20] loss: 0.00137, running_loss: 0.874
Accacy on train_loader set: 99 % [1142/1152]
Accacy on e

Accacy on train_loader set: 98 % [1137/1152]
Accacy on eval_loader set: 71 % [103/144]
[1478,   20] loss: 0.00107, running_loss: 0.686
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 72 % [104/144]
[1479,   20] loss: 0.00160, running_loss: 1.025
Accacy on train_loader set: 98 % [1138/1152]
Accacy on eval_loader set: 77 % [111/144]
[1480,   20] loss: 0.00155, running_loss: 0.991
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 73 % [106/144]
[1481,   20] loss: 0.00096, running_loss: 0.617
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 69 % [100/144]
[1482,   20] loss: 0.00113, running_loss: 0.726
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 67 % [97/144]
[1483,   20] loss: 0.00087, running_loss: 0.555
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 75 % [108/144]
[1484,   20] loss: 0.00116, running_loss: 0.745
Accacy on train_loader set: 98 % [1140/1152]
Accacy on e

In [104]:
max_correct_rate

0.7916666666666666

In [105]:
if __name__ == '__main__':
    for epoch in range(2000):
        train(epoch)
        correct_rate=validate('eval_loader')
        if max_correct_rate < correct_rate:
            max_correct_rate = correct_rate
            saveModel(model_path)

[1,   20] loss: 0.00099, running_loss: 0.635
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 74 % [107/144]
[2,   20] loss: 0.00096, running_loss: 0.612
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 75 % [109/144]
[3,   20] loss: 0.00130, running_loss: 0.833
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 74 % [107/144]
[4,   20] loss: 0.00107, running_loss: 0.687
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 68 % [99/144]
[5,   20] loss: 0.00131, running_loss: 0.839
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 74 % [107/144]
[6,   20] loss: 0.00134, running_loss: 0.858
Accacy on train_loader set: 98 % [1140/1152]
Accacy on eval_loader set: 73 % [106/144]
[7,   20] loss: 0.00087, running_loss: 0.558
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 72 % [105/144]
[8,   20] loss: 0.00080, running_loss: 0.510
Accacy on train_loader set: 99 %

[63,   20] loss: 0.00092, running_loss: 0.589
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 70 % [101/144]
[64,   20] loss: 0.00086, running_loss: 0.548
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 72 % [104/144]
[65,   20] loss: 0.00104, running_loss: 0.668
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 70 % [102/144]
[66,   20] loss: 0.00097, running_loss: 0.620
Accacy on train_loader set: 98 % [1140/1152]
Accacy on eval_loader set: 73 % [106/144]
[67,   20] loss: 0.00171, running_loss: 1.095
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 71 % [103/144]
[68,   20] loss: 0.00095, running_loss: 0.608
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 71 % [103/144]
[69,   20] loss: 0.00112, running_loss: 0.714
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 70 % [101/144]
[70,   20] loss: 0.00126, running_loss: 0.805
Accacy on train_loader 

[125,   20] loss: 0.00102, running_loss: 0.652
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 74 % [107/144]
[126,   20] loss: 0.00136, running_loss: 0.868
Accacy on train_loader set: 98 % [1140/1152]
Accacy on eval_loader set: 72 % [104/144]
[127,   20] loss: 0.00084, running_loss: 0.538
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 70 % [102/144]
[128,   20] loss: 0.00105, running_loss: 0.674
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 77 % [111/144]
[129,   20] loss: 0.00115, running_loss: 0.737
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 75 % [109/144]
[130,   20] loss: 0.00094, running_loss: 0.598
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 77 % [112/144]
[131,   20] loss: 0.00090, running_loss: 0.577
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 73 % [106/144]
[132,   20] loss: 0.00142, running_loss: 0.909
Accacy on train

Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 72 % [104/144]
[187,   20] loss: 0.00103, running_loss: 0.657
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 77 % [111/144]
[188,   20] loss: 0.00140, running_loss: 0.898
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 72 % [105/144]
[189,   20] loss: 0.00082, running_loss: 0.524
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 72 % [104/144]
[190,   20] loss: 0.00105, running_loss: 0.672
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 75 % [108/144]
[191,   20] loss: 0.00123, running_loss: 0.786
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 71 % [103/144]
[192,   20] loss: 0.00097, running_loss: 0.619
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 72 % [105/144]
[193,   20] loss: 0.00110, running_loss: 0.702
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_lo

[248,   20] loss: 0.00103, running_loss: 0.659
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 73 % [106/144]
[249,   20] loss: 0.00094, running_loss: 0.603
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 72 % [105/144]
[250,   20] loss: 0.00081, running_loss: 0.519
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 70 % [101/144]
[251,   20] loss: 0.00087, running_loss: 0.558
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 72 % [104/144]
[252,   20] loss: 0.00078, running_loss: 0.501
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 70 % [101/144]
[253,   20] loss: 0.00075, running_loss: 0.478
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 72 % [104/144]
[254,   20] loss: 0.00076, running_loss: 0.487
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 70 % [101/144]
[255,   20] loss: 0.00128, running_loss: 0.821
Accacy on train

...

[1718,   20] loss: 0.00079, running_loss: 0.507
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 70 % [102/144]
[1719,   20] loss: 0.00105, running_loss: 0.672
Accacy on train_loader set: 98 % [1138/1152]
Accacy on eval_loader set: 75 % [108/144]
[1720,   20] loss: 0.00077, running_loss: 0.491
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 76 % [110/144]
[1721,   20] loss: 0.00109, running_loss: 0.700
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 72 % [104/144]
[1722,   20] loss: 0.00099, running_loss: 0.632
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 75 % [109/144]
[1723,   20] loss: 0.00076, running_loss: 0.486
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 77 % [111/144]
[1724,   20] loss: 0.00121, running_loss: 0.774
Accacy on train_loader set: 98 % [1140/1152]
Accacy on eval_loader set: 74 % [107/144]
[1725,   20] loss: 0.00079, running_loss: 0.504
Accacy 

[1779,   20] loss: 0.00075, running_loss: 0.477
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 75 % [109/144]
[1780,   20] loss: 0.00090, running_loss: 0.574
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 75 % [109/144]
[1781,   20] loss: 0.00083, running_loss: 0.528
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 77 % [111/144]
[1782,   20] loss: 0.00096, running_loss: 0.612
Accacy on train_loader set: 98 % [1139/1152]
Accacy on eval_loader set: 71 % [103/144]
[1783,   20] loss: 0.00094, running_loss: 0.599
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 76 % [110/144]
[1784,   20] loss: 0.00098, running_loss: 0.626
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 70 % [102/144]
[1785,   20] loss: 0.00054, running_loss: 0.346
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 76 % [110/144]
[1786,   20] loss: 0.00091, running_loss: 0.583
Accacy 

[1840,   20] loss: 0.00115, running_loss: 0.739
Accacy on train_loader set: 98 % [1138/1152]
Accacy on eval_loader set: 75 % [109/144]
[1841,   20] loss: 0.00130, running_loss: 0.832
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 77 % [111/144]
[1842,   20] loss: 0.00052, running_loss: 0.333
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 75 % [108/144]
[1843,   20] loss: 0.00079, running_loss: 0.505
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 75 % [108/144]
[1844,   20] loss: 0.00077, running_loss: 0.494
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 77 % [111/144]
[1845,   20] loss: 0.00054, running_loss: 0.349
Accacy on train_loader set: 100 % [1152/1152]
Accacy on eval_loader set: 76 % [110/144]
[1846,   20] loss: 0.00093, running_loss: 0.595
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 78 % [113/144]
[1847,   20] loss: 0.00149, running_loss: 0.954
Accacy

[1901,   20] loss: 0.00103, running_loss: 0.657
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 77 % [111/144]
[1902,   20] loss: 0.00074, running_loss: 0.473
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 70 % [102/144]
[1903,   20] loss: 0.00118, running_loss: 0.755
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 74 % [107/144]
[1904,   20] loss: 0.00097, running_loss: 0.622
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 70 % [102/144]
[1905,   20] loss: 0.00092, running_loss: 0.588
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 77 % [111/144]
[1906,   20] loss: 0.00079, running_loss: 0.506
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 75 % [108/144]
[1907,   20] loss: 0.00151, running_loss: 0.964
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 77 % [111/144]
[1908,   20] loss: 0.00080, running_loss: 0.512
Accacy 

[1962,   20] loss: 0.00059, running_loss: 0.375
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 72 % [104/144]
[1963,   20] loss: 0.00246, running_loss: 1.573
Accacy on train_loader set: 98 % [1133/1152]
Accacy on eval_loader set: 74 % [107/144]
[1964,   20] loss: 0.00073, running_loss: 0.470
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 73 % [106/144]
[1965,   20] loss: 0.00105, running_loss: 0.671
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 73 % [106/144]
[1966,   20] loss: 0.00126, running_loss: 0.808
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 79 % [114/144]
[1967,   20] loss: 0.00093, running_loss: 0.597
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 77 % [112/144]
[1968,   20] loss: 0.00051, running_loss: 0.328
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 76 % [110/144]
[1969,   20] loss: 0.00069, running_loss: 0.439
Accacy 

In [106]:
max_correct_rate

0.8194444444444444

In [107]:
saveData(loss_set,accuracy_train,accuracy_validate)

In [108]:
if __name__ == '__main__':
    for epoch in range(2000):
        train(epoch)
        correct_rate=validate('eval_loader')
        if max_correct_rate < correct_rate:
            max_correct_rate = correct_rate
            saveModel(model_path)

[1,   20] loss: 0.00079, running_loss: 0.508
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 70 % [101/144]
[2,   20] loss: 0.00080, running_loss: 0.509
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 76 % [110/144]
[3,   20] loss: 0.00084, running_loss: 0.540
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 74 % [107/144]
[4,   20] loss: 0.00087, running_loss: 0.558
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 77 % [112/144]
[5,   20] loss: 0.00068, running_loss: 0.434
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 73 % [106/144]
[6,   20] loss: 0.00095, running_loss: 0.608
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 75 % [108/144]
[7,   20] loss: 0.00074, running_loss: 0.477
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 76 % [110/144]
[8,   20] loss: 0.00058, running_loss: 0.368
Accacy on train_loader set: 99 

[63,   20] loss: 0.00115, running_loss: 0.738
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 75 % [108/144]
[64,   20] loss: 0.00086, running_loss: 0.550
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 75 % [109/144]
[65,   20] loss: 0.00128, running_loss: 0.816
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 74 % [107/144]
[66,   20] loss: 0.00065, running_loss: 0.418
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 72 % [104/144]
[67,   20] loss: 0.00089, running_loss: 0.571
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 75 % [108/144]
[68,   20] loss: 0.00065, running_loss: 0.417
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 75 % [108/144]
[69,   20] loss: 0.00080, running_loss: 0.509
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 72 % [105/144]
[70,   20] loss: 0.00066, running_loss: 0.419
Accacy on train_loader 

[125,   20] loss: 0.00104, running_loss: 0.666
Accacy on train_loader set: 98 % [1138/1152]
Accacy on eval_loader set: 74 % [107/144]
[126,   20] loss: 0.00095, running_loss: 0.607
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 74 % [107/144]
[127,   20] loss: 0.00111, running_loss: 0.710
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 75 % [108/144]
[128,   20] loss: 0.00091, running_loss: 0.582
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 70 % [102/144]
[129,   20] loss: 0.00064, running_loss: 0.409
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 75 % [108/144]
[130,   20] loss: 0.00107, running_loss: 0.684
Accacy on train_loader set: 98 % [1138/1152]
Accacy on eval_loader set: 70 % [101/144]
[131,   20] loss: 0.00109, running_loss: 0.695
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 77 % [111/144]
[132,   20] loss: 0.00101, running_loss: 0.644
Accacy on train

Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 79 % [114/144]
[187,   20] loss: 0.00140, running_loss: 0.899
Accacy on train_loader set: 98 % [1138/1152]
Accacy on eval_loader set: 73 % [106/144]
[188,   20] loss: 0.00100, running_loss: 0.643
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 79 % [115/144]
[189,   20] loss: 0.00102, running_loss: 0.650
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 77 % [112/144]
[190,   20] loss: 0.00084, running_loss: 0.540
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 77 % [112/144]
[191,   20] loss: 0.00072, running_loss: 0.459
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 73 % [106/144]
[192,   20] loss: 0.00146, running_loss: 0.937
Accacy on train_loader set: 98 % [1139/1152]
Accacy on eval_loader set: 77 % [111/144]
[193,   20] loss: 0.00080, running_loss: 0.513
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_lo

[248,   20] loss: 0.00077, running_loss: 0.491
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 72 % [104/144]
[249,   20] loss: 0.00067, running_loss: 0.431
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 72 % [104/144]
[250,   20] loss: 0.00108, running_loss: 0.692
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 74 % [107/144]
[251,   20] loss: 0.00106, running_loss: 0.675
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 74 % [107/144]
[252,   20] loss: 0.00070, running_loss: 0.450
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 72 % [105/144]
[253,   20] loss: 0.00073, running_loss: 0.464
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 77 % [112/144]
[254,   20] loss: 0.00086, running_loss: 0.548
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 70 % [102/144]
[255,   20] loss: 0.00069, running_loss: 0.439
Accacy on train

...

[1718,   20] loss: 0.00111, running_loss: 0.713
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 77 % [112/144]
[1719,   20] loss: 0.00070, running_loss: 0.450
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 77 % [111/144]
[1720,   20] loss: 0.00098, running_loss: 0.626
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 79 % [114/144]
[1721,   20] loss: 0.00052, running_loss: 0.334
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 77 % [111/144]
[1722,   20] loss: 0.00098, running_loss: 0.628
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 79 % [114/144]
[1723,   20] loss: 0.00104, running_loss: 0.666
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 81 % [117/144]
[1724,   20] loss: 0.00072, running_loss: 0.461
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 78 % [113/144]
[1725,   20] loss: 0.00107, running_loss: 0.685
Accacy 

[1779,   20] loss: 0.00080, running_loss: 0.511
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 75 % [109/144]
[1780,   20] loss: 0.00057, running_loss: 0.368
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 76 % [110/144]
[1781,   20] loss: 0.00129, running_loss: 0.825
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 76 % [110/144]
[1782,   20] loss: 0.00079, running_loss: 0.506
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 76 % [110/144]
[1783,   20] loss: 0.00067, running_loss: 0.432
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 72 % [104/144]
[1784,   20] loss: 0.00088, running_loss: 0.562
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 77 % [112/144]
[1785,   20] loss: 0.00104, running_loss: 0.666
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 74 % [107/144]
[1786,   20] loss: 0.00064, running_loss: 0.407
Accacy 

[1840,   20] loss: 0.00108, running_loss: 0.690
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 72 % [104/144]
[1841,   20] loss: 0.00077, running_loss: 0.490
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 72 % [105/144]
[1842,   20] loss: 0.00069, running_loss: 0.440
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 71 % [103/144]
[1843,   20] loss: 0.00088, running_loss: 0.564
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 72 % [104/144]
[1844,   20] loss: 0.00110, running_loss: 0.705
Accacy on train_loader set: 98 % [1139/1152]
Accacy on eval_loader set: 76 % [110/144]
[1845,   20] loss: 0.00102, running_loss: 0.651
Accacy on train_loader set: 98 % [1140/1152]
Accacy on eval_loader set: 78 % [113/144]
[1846,   20] loss: 0.00101, running_loss: 0.649
Accacy on train_loader set: 98 % [1140/1152]
Accacy on eval_loader set: 66 % [96/144]
[1847,   20] loss: 0.00160, running_loss: 1.023
Accacy o

[1901,   20] loss: 0.00077, running_loss: 0.492
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 77 % [111/144]
[1902,   20] loss: 0.00069, running_loss: 0.444
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 77 % [112/144]
[1903,   20] loss: 0.00129, running_loss: 0.825
Accacy on train_loader set: 98 % [1138/1152]
Accacy on eval_loader set: 70 % [102/144]
[1904,   20] loss: 0.00066, running_loss: 0.425
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 77 % [112/144]
[1905,   20] loss: 0.00123, running_loss: 0.786
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 81 % [117/144]
[1906,   20] loss: 0.00062, running_loss: 0.399
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 77 % [112/144]
[1907,   20] loss: 0.00043, running_loss: 0.273
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 76 % [110/144]
[1908,   20] loss: 0.00105, running_loss: 0.671
Accacy 

[1962,   20] loss: 0.00097, running_loss: 0.622
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 74 % [107/144]
[1963,   20] loss: 0.00082, running_loss: 0.526
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 70 % [102/144]
[1964,   20] loss: 0.00106, running_loss: 0.678
Accacy on train_loader set: 98 % [1138/1152]
Accacy on eval_loader set: 75 % [108/144]
[1965,   20] loss: 0.00067, running_loss: 0.429
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 72 % [104/144]
[1966,   20] loss: 0.00083, running_loss: 0.534
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 75 % [108/144]
[1967,   20] loss: 0.00092, running_loss: 0.591
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 77 % [111/144]
[1968,   20] loss: 0.00116, running_loss: 0.740
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 73 % [106/144]
[1969,   20] loss: 0.00057, running_loss: 0.368
Accacy 

In [109]:
max_correct_rate

0.8402777777777778

In [110]:
if __name__ == '__main__':
    for epoch in range(2000):
        train(epoch)
        correct_rate=validate('eval_loader')
        if max_correct_rate < correct_rate:
            max_correct_rate = correct_rate
            saveModel(model_path)

[1,   20] loss: 0.00080, running_loss: 0.510
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 70 % [102/144]
[2,   20] loss: 0.00074, running_loss: 0.472
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 75 % [108/144]
[3,   20] loss: 0.00064, running_loss: 0.412
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 77 % [111/144]
[4,   20] loss: 0.00106, running_loss: 0.677
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 80 % [116/144]
[5,   20] loss: 0.00088, running_loss: 0.566
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 75 % [108/144]
[6,   20] loss: 0.00056, running_loss: 0.357
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 72 % [104/144]
[7,   20] loss: 0.00092, running_loss: 0.591
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 81 % [117/144]
[8,   20] loss: 0.00082, running_loss: 0.525
Accacy on train_loader set: 98 

[63,   20] loss: 0.00063, running_loss: 0.401
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 75 % [109/144]
[64,   20] loss: 0.00079, running_loss: 0.508
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 78 % [113/144]
[65,   20] loss: 0.00085, running_loss: 0.543
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 80 % [116/144]
[66,   20] loss: 0.00070, running_loss: 0.446
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 77 % [111/144]
[67,   20] loss: 0.00054, running_loss: 0.344
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 77 % [111/144]
[68,   20] loss: 0.00061, running_loss: 0.388
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 81 % [118/144]
[69,   20] loss: 0.00084, running_loss: 0.536
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 83 % [120/144]
[70,   20] loss: 0.00076, running_loss: 0.486
Accacy on train_loader 

[125,   20] loss: 0.00094, running_loss: 0.603
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 75 % [108/144]
[126,   20] loss: 0.00076, running_loss: 0.487
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 76 % [110/144]
[127,   20] loss: 0.00040, running_loss: 0.255
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 82 % [119/144]
[128,   20] loss: 0.00048, running_loss: 0.307
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 76 % [110/144]
[129,   20] loss: 0.00065, running_loss: 0.416
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 76 % [110/144]
[130,   20] loss: 0.00082, running_loss: 0.527
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 73 % [106/144]
[131,   20] loss: 0.00083, running_loss: 0.534
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 77 % [111/144]
[132,   20] loss: 0.00082, running_loss: 0.523
Accacy on train

Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 77 % [112/144]
[187,   20] loss: 0.00077, running_loss: 0.490
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 78 % [113/144]
[188,   20] loss: 0.00068, running_loss: 0.436
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 73 % [106/144]
[189,   20] loss: 0.00046, running_loss: 0.294
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 77 % [111/144]
[190,   20] loss: 0.00075, running_loss: 0.482
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 78 % [113/144]
[191,   20] loss: 0.00082, running_loss: 0.527
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 72 % [105/144]
[192,   20] loss: 0.00070, running_loss: 0.450
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 77 % [112/144]
[193,   20] loss: 0.00067, running_loss: 0.426
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_lo

[248,   20] loss: 0.00079, running_loss: 0.508
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 74 % [107/144]
[249,   20] loss: 0.00065, running_loss: 0.418
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 75 % [109/144]
[250,   20] loss: 0.00092, running_loss: 0.586
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 75 % [109/144]
[251,   20] loss: 0.00094, running_loss: 0.604
Accacy on train_loader set: 98 % [1140/1152]
Accacy on eval_loader set: 79 % [114/144]
[252,   20] loss: 0.00120, running_loss: 0.768
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 73 % [106/144]
[253,   20] loss: 0.00081, running_loss: 0.517
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 75 % [108/144]
[254,   20] loss: 0.00090, running_loss: 0.577
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 77 % [111/144]
[255,   20] loss: 0.00095, running_loss: 0.609
Accacy on train

...

[1718,   20] loss: 0.00121, running_loss: 0.776
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 73 % [106/144]
[1719,   20] loss: 0.00062, running_loss: 0.394
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 77 % [111/144]
[1720,   20] loss: 0.00087, running_loss: 0.557
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 73 % [106/144]
[1721,   20] loss: 0.00063, running_loss: 0.402
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 75 % [109/144]
[1722,   20] loss: 0.00093, running_loss: 0.597
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 77 % [112/144]
[1723,   20] loss: 0.00085, running_loss: 0.547
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 78 % [113/144]
[1724,   20] loss: 0.00097, running_loss: 0.623
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 75 % [108/144]
[1725,   20] loss: 0.00064, running_loss: 0.411
Accacy 

[1779,   20] loss: 0.00091, running_loss: 0.581
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 78 % [113/144]
[1780,   20] loss: 0.00068, running_loss: 0.434
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 75 % [108/144]
[1781,   20] loss: 0.00125, running_loss: 0.802
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 77 % [112/144]
[1782,   20] loss: 0.00077, running_loss: 0.492
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 79 % [115/144]
[1783,   20] loss: 0.00069, running_loss: 0.444
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 75 % [109/144]
[1784,   20] loss: 0.00070, running_loss: 0.447
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 75 % [108/144]
[1785,   20] loss: 0.00072, running_loss: 0.458
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 72 % [104/144]
[1786,   20] loss: 0.00141, running_loss: 0.904
Accacy 

[1840,   20] loss: 0.00065, running_loss: 0.414
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 78 % [113/144]
[1841,   20] loss: 0.00046, running_loss: 0.292
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 72 % [105/144]
[1842,   20] loss: 0.00074, running_loss: 0.472
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 77 % [111/144]
[1843,   20] loss: 0.00062, running_loss: 0.394
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 77 % [112/144]
[1844,   20] loss: 0.00059, running_loss: 0.379
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 75 % [108/144]
[1845,   20] loss: 0.00071, running_loss: 0.452
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 74 % [107/144]
[1846,   20] loss: 0.00117, running_loss: 0.752
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 75 % [109/144]
[1847,   20] loss: 0.00080, running_loss: 0.515
Accacy 

[1901,   20] loss: 0.00094, running_loss: 0.604
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 75 % [109/144]
[1902,   20] loss: 0.00063, running_loss: 0.406
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 76 % [110/144]
[1903,   20] loss: 0.00055, running_loss: 0.349
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 79 % [114/144]
[1904,   20] loss: 0.00068, running_loss: 0.437
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 77 % [112/144]
[1905,   20] loss: 0.00086, running_loss: 0.549
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 77 % [111/144]
[1906,   20] loss: 0.00055, running_loss: 0.349
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 75 % [109/144]
[1907,   20] loss: 0.00096, running_loss: 0.616
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 75 % [108/144]
[1908,   20] loss: 0.00063, running_loss: 0.406
Accacy 

[1962,   20] loss: 0.00056, running_loss: 0.357
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 77 % [111/144]
[1963,   20] loss: 0.00082, running_loss: 0.524
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 72 % [105/144]
[1964,   20] loss: 0.00065, running_loss: 0.416
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 78 % [113/144]
[1965,   20] loss: 0.00140, running_loss: 0.896
Accacy on train_loader set: 98 % [1140/1152]
Accacy on eval_loader set: 79 % [115/144]
[1966,   20] loss: 0.00082, running_loss: 0.525
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 77 % [111/144]
[1967,   20] loss: 0.00063, running_loss: 0.400
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 71 % [103/144]
[1968,   20] loss: 0.00056, running_loss: 0.355
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 78 % [113/144]
[1969,   20] loss: 0.00084, running_loss: 0.535
Accacy 

In [111]:
max_correct_rate

0.8402777777777778

In [ ]:
saveData(loss_set,accuracy_train,accuracy_validate)

In [35]:
if __name__ == '__main__':
    model=loadModel(model_path)
    validate('eval_loader')

Accacy on eval_loader set: 84 % [121/144]
